<a href="https://colab.research.google.com/github/zeberity123/nvidia-voice-audio-chatbot/blob/main/FastAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://medium.com/@theDrewDag/serve-machine-learning-models-with-fastapi-e329ca3a89c6

In [ ]:
!pip install lightgbm

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()
df = pd.DataFrame(data.data, columns= data.feature_names)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop("MedInc", axis=1)
y = df['MedInc']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(X_train.shape, X_test.shape)


(16512, 7) (4128, 7)


In [ ]:
import lightgbm
import numpy as np
from sklearn import metrics

model = lightgbm.LGBMRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse:.2f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001716 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1583
[LightGBM] [Info] Number of data points in the train set: 16512, number of used features: 7
[LightGBM] [Info] Start training from score 3.880754
Root Mean Squared Error: 0.81


In [ ]:
import pickle

model_filename = 'model.pkl'
with open(model_filename, 'wb') as model_file:
  pickle.dump(model, model_file)

In [ ]:
!pip install fastapi uvicorn

## Creating the API with FastAPI
For the api.py script you need several things:
- A Pydantic class that will hold the data model for the feature set
- A function dedicated to model loading and prediction
- A context manager for the machine learning model life span
- An endpoint to call via browser

### Pydantic to create data model
FastAPI works well with Pydantic and it helps keep things structured and validated.

Pydantic is a library for managing data models and allows to configure and validate them on different properties.

In [ ]:
from pydantic import BaseModel

class FeatureSet(BaseModel):
  HouseAge: float
  AveRooms: float
  AveBedrms: float
  Population: float
  AveOccup: float
  Latitude: float
  Longitude: float

## Model loading and prediction

In [ ]:
def medinc_regressor(x:dict) -> dict:
  with open("model.pkl", 'rb') as model_file:
    loaded_model = pickle.load(model_file)
  x_df = pd.DataFrame(x, index=[0])
  res = loaded_model.predict(x_df)[0]
  return {"prediction": res}
# medinc_regressor is responsible for receiving a feature set x and returning a model response.

## Creating a life span manager for the model

In [ ]:
from fastapi import FastAPI
from contextlib import asynccontextmanager
ml_models = {}

@asynccontextmanager
async def ml_lifespan_manager(app: FastAPI):
  # load the ml model and prediction logic
  ml_models["medinc_regressor"] = medinc_regressor
  yield
  #release the resources + cleanup
  ml_models.clear()

One or machine learning models are shared across various incoming user requests.
There is no association 1 model = 1 user.

The lifespan manager loads the model before the requests are handled but only before the application starts getting requests and not while the code is loading.

## Creating an endpoint to call via browser

In [ ]:
app = FastAPI(lifespan=ml_lifespan_manager)

@app.post("/predict")
async def predict(feature_set: FeatureSet):
  return ml_models["medinc_regressor"](feature_set.dict())

## Start API

In [ ]:
#!pip install google-colab-shell
from google_colab_shell import getshell

getshell()
getshell(height=400)